In [12]:
#from google.colab import drive
import numpy as np
import pandas as pd
import os
import time
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
#drive.mount('/content/drive')

In [3]:
file_path = 'D:\Descargas\datos_gps_curso_ciencia_de_datos'
train_path = file_path + '/train.csv'
val_path = file_path + '/val.csv'
test_path = file_path + '/test.csv'

### set de entrenamiento

In [4]:
train = pd.read_csv(train_path, delimiter = ',')

In [5]:
train['Distancia_Total'] = train['DistanciaInicio2'] - train['DistanciaInicio1']

In [6]:
with open('encoding1.pickle', 'rb') as handle:
    servs_encoding = pickle.load(handle)

In [7]:
train['Servicio'] = train['Servicio'].apply(lambda x: servs_encoding[x])

In [8]:
train.head()

,DistanciaInicio1,DistanciaRuta1,Latitud1,Longitud1,dia_habil1,dia_Monday1,dia_Saturday1,dia_Sunday1,dia_Thursday1,dia_Tuesday1,dia_Wednesday1,DistanciaInicio2,DistanciaRuta2,Latitud2,Longitud2,Servicio,exp_id,Diferencia,Distancia_Total
0,29940,0,-33.409089,-70.568177,1,0,0,0,1,0,0,30011,4,-33.408872,-70.567463,0.059133,ZN-5344_0,30.0,71
1,29940,0,-33.409089,-70.568177,1,0,0,0,1,0,0,30362,6,-33.407944,-70.563855,0.059133,ZN-5344_0,60.0,422
2,29940,0,-33.409089,-70.568177,1,0,0,0,1,0,0,30787,1,-33.407622,-70.559355,0.059133,ZN-5344_0,90.0,847
3,29940,0,-33.409089,-70.568177,1,0,0,0,1,0,0,31105,0,-33.407989,-70.555974,0.059133,ZN-5344_0,120.0,1165
4,29940,0,-33.409089,-70.568177,1,0,0,0,1,0,0,31750,2,-33.408237,-70.549107,0.059133,ZN-5344_0,180.0,1810


In [9]:
X_train = train[['Distancia_Total', 'Servicio']]
y_train = train['Diferencia']

In [10]:
del train

In [11]:
X_train = X_train.values#reshape(-1,1)
X_train.shape

(35818879, 2)

**Regresión Polinomial**

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 

# reg = LinearRegression()
# reg.fit(X_train, y_train)

poly = PolynomialFeatures(degree = 4) 
X_train = poly.fit_transform(X_train) 
#poly.fit(X_train, y_train) 
reg = LinearRegression() 
reg.fit(X_train, y_train)

poly_path = 'polynomial.sav'
pickle.dump(poly, open(poly_path, 'wb'))
reg_path = 'LRegressor.sav'
pickle.dump(reg, open(reg_path, 'wb'))

In [13]:
del poly

In [14]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [15]:
#y_pred = reg.predict(X_train)

y_pred = reg.predict(X_train)

In [16]:
del X_train

In [17]:
r2 = r2_score(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)

print('R2 score: ' + str(r2) )
print('MAE score: ' + str(mae))

R2 score: 0.8348975823107563
MAE score: 186.3835644481618


#### set de validación

In [4]:
poly_path = 'polynomial.sav'
reg_path = 'LRegressor.sav'

poly = pickle.load(open(poly_path, 'rb'))
reg = pickle.load(open(reg_path, 'rb'))

In [5]:
val = pd.read_csv(val_path, delimiter = ',')

In [6]:
val['Distancia_Total'] = val['DistanciaInicio2'] - val['DistanciaInicio1']

In [7]:
with open('encoding1.pickle', 'rb') as handle:
    servs_encoding = pickle.load(handle)

In [8]:
val['Servicio'] = val['Servicio'].apply(lambda x: servs_encoding[x])

In [9]:
val_train = val[['Distancia_Total','Servicio']]
val_target = val['Diferencia']

val_train = val_train.values#.reshape(-1,1)

In [10]:
#y_pred_val = reg.predict(val_train)

y_pred = reg.predict(poly.fit_transform(val_train))

In [14]:
r2_val = r2_score(val_target, y_pred)
mae_val = mean_absolute_error(val_target, y_pred)

print('R2 score: ' + str(r2_val))
print('MAE score: ' + str(mae_val))

R2 score: 0.8328721860385395
MAE score: 187.61020845743298


In [36]:
def eval_range(model, dataset, top=60, bottom=0):
    condition1 = dataset["Diferencia"] >= bottom
    condition2 = dataset["Diferencia"] < top
    total_cond = np.logical_and(condition1, condition2)
    filtered = dataset[total_cond]
    if filtered.shape[0] > 0:
        y_val = filtered["Diferencia"]
        X_val = filtered[['Distancia_Total','Servicio']].values
        y_pred = model.predict(poly.fit_transform(X_val))
        rms = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        return rms, mae, r2
    else:
        return None, None, None

In [38]:
rms, mae, r2 = eval_range(reg, val[['Distancia_Total', 'Diferencia','Servicio']], top=1e20, bottom=0)
print("Errores globales")
print("\tRMS: \t{}".format(rms))
print("\tMAE: \t{}".format(mae))
print("\tR2: \t{}\n".format(r2))


ranges = [[0, 5*60], [5*60, 10*60], [10*60, 20*60], [20*60, 40*60], [40*60, 60*60]]
for bottom, top in ranges:
    rms, mae, r2 = eval_range(reg, val[['Distancia_Total', 'Diferencia','Servicio']], top=top, bottom=bottom)
    print("Errores para el rango {}min a {}min".format(bottom//60, top//60))
    print("\tRMS: \t{}".format(rms))
    print("\tMAE: \t{}".format(mae))
    print("\tR2: \t{}\n".format(r2))

Errores globales
	RMS: 	99584.4398839407
	MAE: 	187.61020845743298
	R2: 	0.8328721860385395

Errores para el rango 0min a 5min
	RMS: 	9624.843200283021
	MAE: 	58.35425066358908
	R2: 	-0.6623966528799987

Errores para el rango 5min a 10min
	RMS: 	47902.98914841107
	MAE: 	151.63420100504382
	R2: 	-5.4646168221558975

Errores para el rango 10min a 20min
	RMS: 	123622.77646940027
	MAE: 	258.3277316538383
	R2: 	-3.2018806891883367

Errores para el rango 20min a 40min
	RMS: 	224462.11690217815
	MAE: 	375.13478357034603
	R2: 	-0.9609597235932097

Errores para el rango 40min a 60min
	RMS: 	595085.1645630051
	MAE: 	627.1675352436276
	R2: 	-4.129262208408644

